In [79]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [80]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
model = keras.models.load_model('model.h5')

In [81]:
import tensorflow.lite as tflite

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp7o0ec0yt/assets


INFO:tensorflow:Assets written to: /tmp/tmp7o0ec0yt/assets
2021-12-02 00:59:18.664635: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2021-12-02 00:59:18.664665: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2021-12-02 00:59:18.664983: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp7o0ec0yt
2021-12-02 00:59:18.666607: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2021-12-02 00:59:18.666628: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmp7o0ec0yt
2021-12-02 00:59:18.672025: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2021-12-02 00:59:18.783102: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /tmp/tmp7o0ec0yt
2021-12-02 00:59:18.800752: I tensorflow/cc/saved_model/loader.cc:283] SavedMode

In [82]:
interpreter = tflite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

print(input_index)
print(output_index)

0
13


In [83]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [84]:
img = download_image('https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg')
img = prepare_image(img,(150,150))

In [85]:
def preprocess_input(x):
    x /= 255
    return x

x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)
X

array([[[[0.7058824 , 0.77254903, 0.7490196 ],
         [0.69411767, 0.7607843 , 0.7294118 ],
         [0.6313726 , 0.69803923, 0.6666667 ],
         ...,
         [0.7647059 , 0.8509804 , 0.8392157 ],
         [0.68235296, 0.74509805, 0.74509805],
         [0.6745098 , 0.76862746, 0.7764706 ]],

        [[0.5254902 , 0.5921569 , 0.5686275 ],
         [0.7372549 , 0.8039216 , 0.78039217],
         [0.60784316, 0.6745098 , 0.6431373 ],
         ...,
         [0.7921569 , 0.8666667 , 0.85882354],
         [0.6313726 , 0.7019608 , 0.6392157 ],
         [0.7882353 , 0.8784314 , 0.85490197]],

        [[0.6862745 , 0.7490196 , 0.7372549 ],
         [0.61960787, 0.6862745 , 0.6627451 ],
         [0.64705884, 0.7137255 , 0.68235296],
         ...,
         [0.6       , 0.65882355, 0.64705884],
         [0.627451  , 0.69411767, 0.654902  ],
         [0.7058824 , 0.79607844, 0.77254903]],

        ...,

        [[0.5803922 , 0.61960787, 0.58431375],
         [0.49019608, 0.5254902 , 0.5058824 ]

In [86]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds

array([[0.770491]], dtype=float32)